In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import urllib.request
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/mohitgupta-omg/Kaggle-SMS-Spam-Collection-Dataset-/master/spam.csv", filename="spam.csv")
data = pd.read_csv('spam.csv',encoding='latin1')


In [ ]:
print('총 샘플의 수 :',len(data))

data[:5]

In [ ]:
del data['Unnamed: 2']
del data['Unnamed: 3']
del data['Unnamed: 4']
data['v1'] = data['v1'].replace(['ham','spam'],[0,1])
data[:5]

In [ ]:
data.info()

In [ ]:
data.isnull().values.any()

In [ ]:
data['v2'].nunique(), data['v1'].nunique()

In [ ]:
data['v1'].value_counts().plot(kind='bar');

In [ ]:
print("ToTal Sample Number:", len(data))

In [ ]:
data['v1'].value_counts().plot(kind='bar');

In [ ]:
print(data.groupby('v1').size().reset_index(name='count'))

In [ ]:
X_data = data['v2']
Y_data = data['v1']
print('Number of Mail {}'.format(len(X_data)))
print('Number of Label {}'.format(len(y_data)))

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_data) # Line 5572 tokenizing
sequences = tokenizer.texts_to_sequences(X_data) # Convert to Index

In [ ]:
print(sequences[:5])